In [ ]:
! pip install requests ffmpeg-python
! pip install deepgram-sdk --upgrade
! pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


In [ ]:
import httpx
import threading
import time
import json

from deepgram import (
    DeepgramClient,
    LiveTranscriptionEvents,
    LiveOptions,
)

# Your Deepgram API key
DEEPGRAM_API_KEY = "INSERT-API-KEY-HERE"

# URL for the realtime streaming audio you would like to transcribe
URL = 'http://stream.live.vc.bbcmedia.co.uk/bbc_world_service'

# Duration for transcription in seconds
TRANSCRIPTION_DURATION = 30

# Set this variable to `True` if you wish only to
# see the transcribed words, like closed captions.
# Set it to `False` if you wish to see the raw JSON responses.
TRANSCRIPT_ONLY = False

def main():
    try:
        # Initialize Deepgram client
        deepgram = DeepgramClient(DEEPGRAM_API_KEY)

        # Create a websocket connection to Deepgram
        dg_connection = deepgram.listen.live.v("1")

        # Define event handlers
        def on_message(self, result, **kwargs):
            if TRANSCRIPT_ONLY:
                sentence = result.channel.alternatives[0].transcript
                if len(sentence) > 0:
                    print(f"Speaker: {sentence}")
            else:
                # Extract relevant information from the LiveResultResponse object
                response_dict = {
                    "type": result.type,
                    "channel": {
                        "alternatives": [
                            {
                                "transcript": alt.transcript
                            } for alt in result.channel.alternatives
                        ]
                    }
                }
                print(json.dumps(response_dict, indent=4))


        def on_metadata(self, metadata, **kwargs):
            print(f"\n\n{metadata}\n\n")

        def on_error(self, error, **kwargs):
            print(f"\n\n{error}\n\n")

        # Register event handlers
        dg_connection.on(LiveTranscriptionEvents.Transcript, on_message)
        dg_connection.on(LiveTranscriptionEvents.Metadata, on_metadata)
        dg_connection.on(LiveTranscriptionEvents.Error, on_error)

        # Configure Deepgram options for live transcription
        options = LiveOptions(
            model="nova-2",
            language="en-US",
            smart_format=True,
        )

        # Start the connection
        dg_connection.start(options)

        # Create a lock and a flag for thread synchronization
        lock_exit = threading.Lock()
        exit = False

        # Define a thread that streams the audio and sends it to Deepgram
        def myThread():
            start_time = time.time()
            with httpx.stream("GET", URL) as r:
                for data in r.iter_bytes():
                    if time.time() - start_time >= TRANSCRIPTION_DURATION:
                        break
                    dg_connection.send(data)

        # Start the thread
        myHttp = threading.Thread(target=myThread)
        myHttp.start()

        # Wait for the specified duration
        myHttp.join(TRANSCRIPTION_DURATION)

        # Set the exit flag to True to stop the thread
        lock_exit.acquire()
        exit = True
        lock_exit.release()

        # Close the connection to Deepgram
        dg_connection.finish()

        print("Finished")

    except Exception as e:
        print(f"Could not open socket: {e}")
        return


if __name__ == "__main__":
    main()

<ipython-input-4-635f38cab067>:32: DeprecatedWarning: live is deprecated as of 3.4.0 and will be removed in 4.0.0. deepgram.listen.live is deprecated. Use deepgram.listen.websocket instead.
  dg_connection = deepgram.listen.live.v("1")


{
    "type": "Results",
    "channel": {
        "alternatives": [
            {
                "transcript": "Mackenzie letter."
            }
        ]
    }
}
{
    "type": "Results",
    "channel": {
        "alternatives": [
            {
                "transcript": "And Ryan Krausser then looking to win a 3rd Olympic medal, which is just remarkable."
            }
        ]
    }
}
{
    "type": "Results",
    "channel": {
        "alternatives": [
            {
                "transcript": "A"
            }
        ]
    }
}
{
    "type": "Results",
    "channel": {
        "alternatives": [
            {
                "transcript": "I like the fact that he's realizing he sets his own training programs, and he's realized"
            }
        ]
    }
}
{
    "type": "Results",
    "channel": {
        "alternatives": [
            {
                "transcript": "I might have to tailor these a little bit. They're now at my age."
            }
        ]
    }
}
{
    "typ

In [ ]:
from deepgram import DeepgramClient, PrerecordedOptions, FileSource
import os

# Deepgram API key
DG_KEY = "INSERT-API-KEY-HERE"

# Path to the local audio file
AUDIO_FILE_PATH = "story.mp3"

# Path to save the transcript JSON file
TRANSCRIPT_FILE = "transcript.json"

def main():
    try:
        # STEP 1: Create a Deepgram client using the API key
        deepgram = DeepgramClient(DG_KEY)

        # Read the local audio file
        if os.path.exists(AUDIO_FILE_PATH):
            with open(AUDIO_FILE_PATH, "rb") as audio_file:
                buffer_data = audio_file.read()
        else:
            print("Audio file not found")
            return

        payload: FileSource = {
            "buffer": buffer_data,
        }

        # STEP 2: Configure Deepgram options for audio analysis
        options = PrerecordedOptions(
            model="nova-2",
            smart_format=True,
        )

        # STEP 3: Call the transcribe_file method with the text payload and options
        response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)

        # STEP 4: Write the response JSON to a file
        with open(TRANSCRIPT_FILE, "w") as transcript_file:
            transcript_file.write(response.to_json(indent=4))

        print("Transcript JSON file generated successfully.")

    except Exception as e:
        print(f"Exception: {e}")

if __name__ == "__main__":
    main()

<ipython-input-2-e18575956610>:37: DeprecatedWarning: prerecorded is deprecated as of 3.4.0 and will be removed in 4.0.0. deepgram.listen.prerecorded is deprecated. Use deepgram.listen.rest instead.
  response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)


Transcript JSON file generated successfully.


In [ ]:
import asyncio
import json
import logging
import random
import time
import os
from typing import Callable, Dict, List, Tuple
from deepgram import DeepgramClient, PrerecordedOptions, FileSource

DG_KEY = "INSERT-API-KEY-HERE"
AUDIO_FILE_PATH = "story.mp3"
TRANSCRIPT_FILE = "transcript.json"

class AIBenchRunner:
    def __init__(self, fn: Callable, load: int, seed: int = 42):
        self.fn = fn
        self.load = load
        self.ttft = []
        self.end_to_end_latency = []
        self.cold_start = 0
        self.failed_queries = 0
        self.prompt_queue = asyncio.Queue()
        self.results_queue = asyncio.Queue()
        random.seed(seed)

    async def __call__(self) -> Dict:
        self.cold_start = await self.check_coldstart(threshold=15)
        concurrent_requests = []
        for audio_data in self.prepare_prompts():
            await self.prompt_queue.put(audio_data)
        for _ in range(self.load):
            concurrent_requests.append(asyncio.create_task(self.compute_metrics()))
        await asyncio.gather(*concurrent_requests)
        await self.unpack_metrics()
        return self.as_dict()

    @property
    def itl(self) -> List[float]:
        return [
            (e2e_lat - ttft) / 1
            for e2e_lat, ttft in zip(
                self.end_to_end_latency,
                self.ttft,
            )
        ]

    @property
    def output_tks_per_sec(self) -> List[float]:
        return [1000 / i for i in self.itl]

    def as_dict(self) -> Dict:
        return {
            "load": self.load,
            "ttft": self.ttft,
            "e2e_latency": self.end_to_end_latency,
            "itl": self.itl,
            "cold_start": self.cold_start,
            "failed_queries": self.failed_queries,
        }

    async def unpack_metrics(self):
        while not self.results_queue.empty():
            req_result: dict = await self.results_queue.get()
            print(req_result)
            self.ttft.append(req_result["ttft"])
            self.end_to_end_latency.append(req_result["e2e_latency"])
            self.failed_queries += req_result["failed_queries"]
            self.results_queue.task_done()

    @staticmethod
    def _get_audio_data() -> bytes:
        if os.path.exists(AUDIO_FILE_PATH):
            with open(AUDIO_FILE_PATH, "rb") as audio_file:
                return audio_file.read()
        else:
            raise FileNotFoundError("Audio file not found")

    def prepare_prompts(self) -> List[bytes]:
        audio_data = self._get_audio_data()
        return [audio_data] * self.load

    async def compute_metrics(self) -> None:
        audio_data = await self.prompt_queue.get()
        completions = []
        metrics_dict = {}

        deepgram = DeepgramClient(DG_KEY)
        payload: FileSource = {"buffer": audio_data}
        options = PrerecordedOptions(model="nova-2", smart_format=True)

        start_time = time.perf_counter()
        try:
            result = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)
        except Exception as e:
            print(f"exception: {e}")
            metrics_dict["failed_queries"] = 1
            self.results_queue.put_nowait(metrics_dict)
            self.prompt_queue.task_done()
            return

        end_time = time.perf_counter()
        metrics_dict["failed_queries"] = 0

        content = result.to_dict()
        metrics_dict["ttft"] = (end_time - start_time) * 1000  # Adjust as needed
        metrics_dict["e2e_latency"] = (end_time - start_time) * 1000
        await self.results_queue.put(metrics_dict)
        self.prompt_queue.task_done()

    async def check_coldstart(self, threshold: float) -> float:
        deepgram = DeepgramClient(DG_KEY)
        audio_data = self._get_audio_data()
        payload: FileSource = {"buffer": audio_data}
        options = PrerecordedOptions(model="nova-2", smart_format=True)

        start_time = time.perf_counter()
        try:
            result = await deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)
        except Exception as e:
            logging.warning("Run during cold start failed")
            return 0
        end_time = time.perf_counter()

        cold_start = end_time - start_time
        if cold_start > threshold:
            return cold_start
        return 0



# Example usage
async def main():
    runner = AIBenchRunner(fn=None, load=5)  # fn is not used in this context
    metrics = await runner()
    print(metrics)

if __name__ == "__main__":
    await main()


<ipython-input-6-e49de1edbfd4>:118: DeprecatedWarning: prerecorded is deprecated as of 3.4.0 and will be removed in 4.0.0. deepgram.listen.prerecorded is deprecated. Use deepgram.listen.rest instead.
  result = await deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)
<ipython-input-6-e49de1edbfd4>:93: DeprecatedWarning: prerecorded is deprecated as of 3.4.0 and will be removed in 4.0.0. deepgram.listen.prerecorded is deprecated. Use deepgram.listen.rest instead.
  result = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)


{'failed_queries': 0, 'ttft': 2670.3641200001584, 'e2e_latency': 2670.3641200001584}
{'failed_queries': 0, 'ttft': 2515.0435440000365, 'e2e_latency': 2515.0435440000365}
{'failed_queries': 0, 'ttft': 2585.029774999839, 'e2e_latency': 2585.029774999839}
{'failed_queries': 0, 'ttft': 2799.0611500001705, 'e2e_latency': 2799.0611500001705}
{'failed_queries': 0, 'ttft': 2214.903422999896, 'e2e_latency': 2214.903422999896}
{'load': 5, 'ttft': [2670.3641200001584, 2515.0435440000365, 2585.029774999839, 2799.0611500001705, 2214.903422999896], 'e2e_latency': [2670.3641200001584, 2515.0435440000365, 2585.029774999839, 2799.0611500001705, 2214.903422999896], 'itl': [0.0, 0.0, 0.0, 0.0, 0.0], 'cold_start': 0, 'failed_queries': 0}
